In [1]:
import pandas as pd
import glob
import numpy as np
import os
import sys

# Get the parent directory (ManifoldSPC)
project_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Add the project directory to sys.path
sys.path.append(project_dir)
print(project_dir)

from scripts2.DR_methods import *
from scripts2.Filters import *
from scripts2.functions import *
from scripts2.TE_functions_v6 import parse_filename
from ControlChart.control_chart import DFEWMA
from ControlChart.control_chart import DME
# from scripts.YSL23_test2 import YSL23 as MF
from scripts2.functions import *

c:\Users\ibura\Manifold SPC Paper v3


In [2]:
xnsadd_stddev = np.array([
    0.01,   # xmeasadd[0]
    0.01,   # xmeasadd[1]
    0.01,   # xmeasadd[2]
    0.01,   # xmeasadd[3]
    0.01,   # xmeasadd[4]
    0.125,  # xmeasadd[5]
    0.01,   # xmeasadd[6]
    0.125,  # xmeasadd[7]
    0.01,   # xmeasadd[8]
    0.01,   # xmeasadd[9]
    0.25,   # xmeasadd[10]
    0.1,    # xmeasadd[11]
    0.25,   # xmeasadd[12]
    0.1,    # xmeasadd[13]
    0.25,   # xmeasadd[14]
    0.025,  # xmeasadd[15]
    0.25,   # xmeasadd[16]
    0.1,    # xmeasadd[17]
    0.25,   # xmeasadd[18]
    0.1,    # xmeasadd[19]
    0.25,   # xmeasadd[20]
    0.025,  # xmeasadd[21]
    0.25,   # xmeasadd[22]
    0.1,    # xmeasadd[23]
    0.25,   # xmeasadd[24]
    0.1,    # xmeasadd[25]
    0.25,   # xmeasadd[26]
    0.025,  # xmeasadd[27]
    0.25,   # xmeasadd[28]
    0.1,    # xmeasadd[29]
    0.25,   # xmeasadd[30]
    0.1     # xmeasadd[31]
])

xns_stddev = np.array([
    0.0012,  # xmeas[0]
    18.0,    # xmeas[1]
    22.0,    # xmeas[2]
    0.05,    # xmeas[3]
    0.2,     # xmeas[4]
    0.21,    # xmeas[5]
    0.3,     # xmeas[6]
    0.5,     # xmeas[7]
    0.01,    # xmeas[8]
    0.0017,  # xmeas[9]
    0.01,    # xmeas[10]
    1.0,     # xmeas[11]
    0.3,     # xmeas[12]
    0.125,   # xmeas[13]
    1.0,     # xmeas[14]
    0.3,     # xmeas[15]
    0.115,   # xmeas[16]
    0.01,    # xmeas[17]
    1.15,    # xmeas[18]
    0.2,     # xmeas[19]
    0.01,    # xmeas[20]
    0.01,    # xmeas[21]
    0.25,    # xmeas[22]
    0.1,     # xmeas[23]
    0.25,    # xmeas[24]
    0.1,     # xmeas[25]
    0.25,    # xmeas[26]
    0.025,   # xmeas[27]
    0.25,    # xmeas[28]
    0.1,     # xmeas[29]
    0.25,    # xmeas[30]
    0.1,     # xmeas[31]
    0.25,    # xmeas[32]
    0.025,   # xmeas[33]
    0.05,    # xmeas[34]
    0.05,    # xmeas[35]
    0.01,    # xmeas[36]
    0.01,    # xmeas[37]
    0.01,    # xmeas[38]
    0.5,     # xmeas[39]
    0.5      # xmeas[40]
])

keep_mask = np.array([
    0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
    1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
    1, 1, 0, 1, 1, 0
])

obs_order = np.hstack([
    np.array([1]),
    xnsadd_stddev,
    xns_stddev,
    np.ones(12)
])

keep_mask_temp = np.hstack([
    np.ones(30),
    np.zeros(9)
]).astype(bool) 

obs_order = obs_order[keep_mask==1]
# print(obs_order.shape)
obs_order = np.tile(obs_order, (10, 1)).reshape(-1)
keep_mask_temp = np.tile(keep_mask_temp,(10, 1)).reshape(-1)

In [3]:
d = np.loadtxt("dataIC.csv", delimiter=",")
d = d[:20000]/obs_order
d = d[:,keep_mask_temp]


In [4]:
n_warm_up = 5000
n_manifold = 700 + n_warm_up
n_var = 400
n_ic = n_manifold+n_var-n_warm_up
n_test = 100

manifold_data = d[n_warm_up:n_manifold].copy()
ic_data = d[n_warm_up:n_ic].copy()
max_manifold = np.max(np.abs(manifold_data))
print(max_manifold)
manifold_data = manifold_data/max_manifold
ic_data = ic_data/max_manifold
var_data = d[n_manifold:n_manifold+n_var].copy()/max_manifold
test_data = d[n_manifold+n_var:n_manifold+n_var+n_test].copy()/max_manifold
true_sigma = 1/max_manifold
print(true_sigma)

ic_mean = np.mean(d/max_manifold,axis=0)
ic_std = np.std(d/max_manifold,axis=0)


12294.3349761742
8.133827506229084e-05


In [5]:
ic_mean = np.mean(d[:,:30]/max_manifold,axis=0)
ic_std = np.std(d[:,:30]/max_manifold,axis=0)
print(ic_mean)
print(ic_std)

[3.66022095e-01 3.66022070e-01 3.66022693e-01 3.66022742e-01
 2.84683986e-01 5.31304459e-02 3.25353146e-01 2.18798292e-02
 1.80832586e-02 1.65188357e-02 1.64191264e-02 1.50254195e-02
 1.31848068e-02 1.84357219e-02 7.59158011e-01 1.05748212e-02
 9.99647016e-01 1.01383240e-02 7.48482321e-01 4.06631982e-03
 7.33273849e-01 1.64389385e-02 4.06690199e-03 9.03410048e-01
 1.61911087e-02 5.42054164e-01 8.26140570e-07 1.11032176e-01
 8.33442434e-01 7.51232909e-01]
[8.08511320e-05 8.13454022e-05 8.20397890e-05 8.12428877e-05
 8.13356863e-05 1.17577895e-04 8.16856463e-05 1.27233735e-03
 3.86759215e-04 8.57832650e-05 8.77932603e-05 8.44339760e-05
 8.25819234e-05 8.23006925e-05 2.67795800e-04 8.12398970e-05
 8.85728013e-05 4.71613323e-04 5.67356024e-04 8.21500558e-05
 2.66793051e-04 8.22607509e-05 8.29022638e-05 3.08626045e-04
 8.17515408e-05 3.80167858e-04 8.08347707e-05 1.45636697e-04
 1.60743294e-04 1.22170788e-03]


In [6]:
te_dr = os.getcwd()
pickle_path = os.path.join(te_dr, r"TEpkl")
data_path = os.path.join(te_dr, r"Data")
from pathlib import Path

keep_mask_small = keep_mask_temp[:int(len(keep_mask_temp) / 10)]
obs_order_small = obs_order[:int(len(obs_order) / 10)]
def extract_mean(file):
    d = pd.read_csv(file, header=None).to_numpy()[:,1:]
    d = d/obs_order_small
    d = d/max_manifold
    d = d[:,keep_mask_small]
    d_oc = d[1200:]
    return np.mean(d_oc, axis=0)

def shift_size(data_path,fault_no,amplitude):
    csv_files = list(Path(data_path).glob("*.csv"))
    data = [parse_filename(file) for file in csv_files]
    df = pd.DataFrame(data)

    df["fault"] = df["fault"].astype(int)
    df["amplitude"] = df["amplitude"].astype(float)
    df["RunID"] = df["RunID"].astype(int)
    df = df.sort_values(by="RunID")
    faulty_files = df[(df["fault"] == fault_no) & (df["amplitude"] == amplitude)]
    faulty_files = faulty_files[:100]
    oc_mean = []
    for _, row in faulty_files.iterrows():
        mean_vals = extract_mean(row["path"])
        oc_mean.append(mean_vals)

    if oc_mean:
        return np.max(np.abs(np.mean(oc_mean, axis=0)-ic_mean)/ic_std)
    else:
        return None



In [7]:
te_dr = os.getcwd()
pickle_path = os.path.join(te_dr, r"TEpkl")
data_path = os.path.join(te_dr, r"Data")
from pathlib import Path

keep_mask_small = keep_mask_temp[:int(len(keep_mask_temp) / 10)]
obs_order_small = obs_order[:int(len(obs_order) / 10)]
def extract_mean_ic(file):
    d = pd.read_csv(file, header=None).to_numpy()[:,1:]
    d = d/obs_order_small
    d = d/max_manifold
    d = d[:,keep_mask_small]
    d_oc = d[:1200]
    return np.mean(d_oc, axis=0)

def shift_size_ic(data_path,fault_no,amplitude):
    csv_files = list(Path(data_path).glob("*.csv"))
    data = [parse_filename(file) for file in csv_files]
    df = pd.DataFrame(data)

    df["fault"] = df["fault"].astype(int)
    df["amplitude"] = df["amplitude"].astype(float)
    df["RunID"] = df["RunID"].astype(int)
    df = df.sort_values(by="RunID")
    faulty_files = df[(df["fault"] == fault_no) & (df["amplitude"] == amplitude)]
    faulty_files = faulty_files[:100]
    oc_mean = []
    for _, row in faulty_files.iterrows():
        mean_vals = extract_mean_ic(row["path"])
        oc_mean.append(mean_vals)

    if oc_mean:
        return np.max(np.abs(np.mean(oc_mean, axis=0)-ic_mean)/ic_std)
    else:
        return None



In [ ]:
import os
import re
import pandas as pd
from pathlib import Path

# ==== Keep these utility functions unchanged ====


def extract_mean(file):
    d = pd.read_csv(file, header=None).to_numpy()[:, 1:]
    d = d[:, keep_mask_temp]  # Assume keep_mask_temp is globally defined
    d_oc = d[1200:]
    return np.mean(d_oc, axis=0)

# ==== Shift Size Calculation Function ====

def shift_size(data_path, fault_no, amplitude):
    csv_files = list(Path(data_path).glob("*.csv"))
    data = [parse_filename(file) for file in csv_files]
    df = pd.DataFrame(data)

    df["fault"] = df["fault"].astype(int)
    df["amplitude"] = df["amplitude"].astype(float)
    df["RunID"] = df["RunID"].astype(int)
    df = df.sort_values(by="RunID")

    faulty_files = df[(df["fault"] == fault_no) & (df["amplitude"] == amplitude)]
    faulty_files = faulty_files[:100]

    oc_mean = []
    for _, row in faulty_files.iterrows():
        mean_vals = extract_mean(row["path"])
        oc_mean.append(mean_vals)

    if oc_mean:
        return np.max(np.abs(np.mean(oc_mean, axis=0) - ic_mean) / ic_std)
    else:
        return None

# ==== ARL Processing Function ====

def process_experiment(experiment_name, base_path):
    experiment_folder = os.path.join(base_path, f"{experiment_name}", "RunLengths")
    methods = ["LPP", "NPE", "MF", "PCA", "AM"]

    results = []

    for method in methods:
        method_path = os.path.join(experiment_folder, method)

        if not os.path.exists(method_path):
            print(f"Skipping missing folder: {method_path}")
            continue

        data = []

        for file in os.listdir(method_path):
            if file.endswith(".csv"):
                amplitude, rl = extract_values(file)
                if amplitude is not None:
                    data.append((amplitude, rl))

        if data:
            df = pd.DataFrame(data, columns=["Amplitude", "RL"])
            stats = df.groupby("Amplitude")["RL"].agg(["mean", "std"]).reset_index()
            stats.rename(columns={"mean": "RL", "std": "Std"}, inplace=True)
            stats["Method"] = method
            results.append(stats)

    if results:
        final_df = pd.concat(results, ignore_index=True)
        return final_df
    else:
        return pd.DataFrame(columns=["Amplitude", "RL", "Std", "Method"])


# ==== Main Loop for Faults ====

def collect_fault_tables(base_path, data_path):
    all_results = []

    fault_list = list(range(1, 6)) + list(range(7, 29))  # 1–5 and 7–28
    for fault_no in fault_list:
        experiment_name = f"Exptestf{fault_no}_100"
        print(f"Processing Fault {fault_no}...")

        # Step 1: Get ARL data
        arl_df = process_experiment(experiment_name, base_path)

        if arl_df.empty:
            print(f"No data for Fault {fault_no}")
            continue

        # Step 2: Get shift size
        shift = shift_size(data_path, fault_no, amplitude=1.0)

        # Step 3: Add FaultNo and ShiftSize to every row
        arl_df["Fault"] = fault_no
        arl_df["ShiftSize"] = shift

        all_results.append(arl_df)

    if all_results:
        final_table = pd.concat(all_results, ignore_index=True)
        return final_table
    else:
        return pd.DataFrame(columns=["Fault", "Amplitude", "RL", "Std", "Method", "ShiftSize"])

# ==== Usage ====

base_path = r"C:\Users\ibura\Manifold SPC Paper v3\TE Simulations\Experiments"
data_path = r"C:\Users\ibura\Manifold SPC Paper v3\TE Simulations\Data"

final_result = collect_fault_tables(base_path, data_path)
print(final_result)


In [90]:

import os
import re
import pandas as pd
from pathlib import Path

def extract_values(filename):
    match = re.search(r"amplitude-([\d\.-]+)--rl-(\d+)", filename)
    if match:
        amplitude = float(match.group(1))
        rl = int(match.group(2))
        return amplitude, rl
    return None, None


def process_experiment(experiment_name, base_path):
    experiment_folder = os.path.join(base_path, f"{experiment_name}", "RunLengths")
    methods = ["LPP", "NPE", "MF", "PCA"]

    results = []
    for method in methods:
        method_path = os.path.join(experiment_folder, method)
        if not os.path.exists(method_path):
            print(f"Skipping missing folder: {method_path}")
            continue

        data = []
        for file in os.listdir(method_path):
            if file.endswith(".csv"):
                amplitude, rl = extract_values(file)
                if amplitude is not None:
                    data.append((amplitude, rl))

        if data:
            df = pd.DataFrame(data, columns=["Amplitude", "RL"])
            stats = df.groupby("Amplitude")["RL"].agg(["mean", "std"]).reset_index()
            stats.rename(columns={"mean": "RL", "std": "Std"}, inplace=True)
            stats["Method"] = method
            results.append(stats)

    if results:
        return pd.concat(results, ignore_index=True)
    else:
        return pd.DataFrame(columns=["Amplitude", "RL", "Std", "Method"])


# === Final loop to generate one ARL+Shift table per fault ===
def collect_fault_tables(base_path, data_path):
    all_results = []
    fault_list = list(range(1, 6)) + list(range(7, 29))  # 1–5 and 7–28

    for fault_no in fault_list:
        print(f"Processing Fault {fault_no}...")
        experiment_name = f"Exptestf{fault_no}_100"

        # Step 1: ARL Table
        arl_df = process_experiment(experiment_name, base_path)
        if arl_df.empty:
            print(f"No ARL data for fault {fault_no}")
            continue

        # Step 2: Shift Size
        shift = shift_size(data_path, fault_no, amplitude=1.0)

        # Step 3: Merge
        arl_df["Fault"] = fault_no
        arl_df["ShiftSize"] = shift

        all_results.append(arl_df)

    if all_results:
        return pd.concat(all_results, ignore_index=True)
    else:
        return pd.DataFrame(columns=["Fault", "Amplitude", "RL", "Std", "Method", "ShiftSize"])


# === Usage ===
base_path = r"C:\Users\ibura\Manifold SPC Paper v3\TE Simulations\Experiments"
final_result = collect_fault_tables(base_path, data_path)
print(final_result)

Processing Fault 1...
Processing Fault 2...
Processing Fault 3...
Processing Fault 4...
Processing Fault 5...
Processing Fault 7...
Processing Fault 8...
Processing Fault 9...
Processing Fault 10...
Processing Fault 11...
Processing Fault 12...
Processing Fault 13...
Processing Fault 14...
Processing Fault 15...
Processing Fault 16...
Processing Fault 17...
Processing Fault 18...
Processing Fault 19...
Processing Fault 20...
Processing Fault 21...
Processing Fault 22...
Processing Fault 23...
Processing Fault 24...
Processing Fault 25...
Processing Fault 26...
Processing Fault 27...
Processing Fault 28...
     Amplitude     RL        Std Method  Fault  ShiftSize
0          1.0   5.75   1.866098    LPP      1  85.052531
1          1.0   5.40   1.814643    NPE      1  85.052531
2          1.0   5.54   1.816979     MF      1  85.052531
3          1.0   5.97   1.898724    PCA      1  85.052531
4          1.0   9.11   5.283661    LPP      2  17.576024
..         ...    ...        ...    ...

In [92]:
def format_arl_table(df):
    # Keep only required methods
    methods = ["LPP", "MF", "NPE", "PCA"]
    df = df[df["Method"].isin(methods)]

    # Format ARL with std in parentheses
    df["ARL_Formatted"] = df.apply(lambda row: f'{row["RL"]:.1f} ({row["Std"]:.1f})', axis=1)

    # Pivot the table: Fault, Amplitude, ShiftSize as index, methods as columns
    pivot_df = df.pivot_table(index=["Fault", "Amplitude", "ShiftSize"],
                              columns="Method",
                              values="ARL_Formatted",
                              aggfunc="first").reset_index()

    # Optional: reorder columns
    method_order = ["LPP", "MF", "NPE", "PCA"]
    pivot_df = pivot_df[["Fault", "Amplitude", "ShiftSize"] + [m for m in method_order if m in pivot_df.columns]]

    return pivot_df


In [93]:
# Step 1: Collect full raw result table
final_result = collect_fault_tables(base_path, data_path)

# Step 2: Format it as described
summary_table = format_arl_table(final_result)

# Step 3: View or save
print(summary_table)
# summary_table.to_csv("ARL_summary_table.csv", index=False)


Processing Fault 1...
Processing Fault 2...
Processing Fault 3...
Processing Fault 4...
Processing Fault 5...
Processing Fault 7...
Processing Fault 8...
Processing Fault 9...
Processing Fault 10...
Processing Fault 11...
Processing Fault 12...
Processing Fault 13...
Processing Fault 14...
Processing Fault 15...
Processing Fault 16...
Processing Fault 17...
Processing Fault 18...
Processing Fault 19...
Processing Fault 20...
Processing Fault 21...
Processing Fault 22...
Processing Fault 23...
Processing Fault 24...
Processing Fault 25...
Processing Fault 26...
Processing Fault 27...
Processing Fault 28...
Method  Fault  Amplitude   ShiftSize          LPP           MF          NPE  \
0           1        1.0   85.052531    5.8 (1.9)    5.5 (1.8)    5.4 (1.8)   
1           2        1.0   17.576024    9.1 (5.3)    8.2 (4.1)    9.4 (5.1)   
2           3        1.0  497.883546    1.6 (0.5)    2.0 (0.7)    1.7 (0.4)   
3           4        1.0  497.935977    1.7 (0.5)    1.9 (0.7)    1.7 (

In [94]:
# Step 3: View or save
print(summary_table)
# summary_table.to_csv("ARL_summary_table.csv", index=False)

Method  Fault  Amplitude   ShiftSize          LPP           MF          NPE  \
0           1        1.0   85.052531    5.8 (1.9)    5.5 (1.8)    5.4 (1.8)   
1           2        1.0   17.576024    9.1 (5.3)    8.2 (4.1)    9.4 (5.1)   
2           3        1.0  497.883546    1.6 (0.5)    2.0 (0.7)    1.7 (0.4)   
3           4        1.0  497.935977    1.7 (0.5)    1.9 (0.7)    1.7 (0.5)   
4           5        1.0  495.802301    1.7 (0.5)    2.1 (0.7)    1.6 (0.5)   
5           7        1.0    0.245283    3.2 (1.1)    2.3 (0.7)    3.0 (1.0)   
6           8        1.0    1.627202   12.2 (8.4)   13.7 (7.6)   12.6 (8.8)   
7           9        1.0    7.385667    8.3 (4.4)    7.7 (4.4)    7.3 (4.3)   
8          10        1.0    3.350745   11.5 (8.1)   12.1 (8.8)   11.3 (8.4)   
9          11        1.0   15.427773    4.9 (2.4)    5.0 (2.5)    4.9 (2.2)   
10         12        1.0    9.519076    5.2 (2.6)    5.5 (2.5)    5.8 (2.6)   
11         13        1.0   21.706332  16.6 (12.4)  1

In [ ]:
# Step 3: View or save
print(summary_table)
# summary_table.to_csv("ARL_summary_table.csv", index=False)

In [95]:
summary_table.to_csv("ARL_summary_table.csv", index=False)

In [8]:
from sklearn.decomposition import PCA
n_dim = 22
n_neighbors = 15
pca = PCA(n_components=n_dim)
pca_ic = pca.fit_transform(manifold_data)

lpp = LocalityPreservingProjection(n_components=n_dim,n_neighbors=n_neighbors,weight="heat")
lpp_ic = lpp.fit_transform(manifold_data)

npe = NeighborhoodPreservingEmbedding(n_components=n_dim,n_neighbors=n_neighbors)
npe_ic = npe.fit_transform(manifold_data)

In [9]:
P_lpp = lpp.projection_.T
P_npe = npe.components_.T
P_pca = pca.components_

In [10]:
te_dr = os.getcwd()
faults_folder = "faults"
print(te_dr)
faults_path = os.path.join(te_dr,faults_folder)
csv_files = [f for f in os.listdir(faults_path) if f.endswith('.csv')]
print(faults_path)

c:\Users\ibura\Manifold SPC Paper v3\TE Simulations
c:\Users\ibura\Manifold SPC Paper v3\TE Simulations\faults


In [ ]:
def output_the_mean(faults_path,fault_no,amplitude):
    
    csv_files = list(Path(faults_path).glob("*.csv"))
    data = [parse_filename(file) for file in csv_files]
    df = pd.DataFrame(data)

    df["fault"] = df["fault"].astype(int)
    df["amplitude"] = df["amplitude"].astype(float)
    df["RunID"] = df["RunID"].astype(int)
    df = df.sort_values(by="RunID")

    faulty_files = df[(df["fault"] == fault_no) & (df["amplitude"] == amplitude)]
    #print(faulty_files)

    return extract_mean_oc_with_padding(faulty_files.iloc[0]["path"])


keep_mask_small = keep_mask_temp[:int(len(keep_mask_temp) / 10)]
obs_order_small = obs_order[:int(len(obs_order) / 10)]
def extract_mean_oc_with_padding(file):
    d = pd.read_csv(file, header=None).to_numpy()[:,1:]
    d = d/obs_order_small
    d = d/max_manifold
    d = d[:,keep_mask_small]
    d_oc = d[2000:]
    vec = np.mean(d_oc, axis=0)
    vec = vec - ic_mean
    vec = np.pad(vec, (0, max(0, 300 - len(vec))), mode='constant')
    norm = np.linalg.norm(vec)
    if norm > 0:
        vec = vec
    return vec
    

In [104]:
output_the_mean(faults_path,14,1)

np.float64(8.895479597392774e-05)

In [42]:
import numpy as np
from scipy.linalg import null_space

def decompose_vector_by_projection(P, v):
    """
    P: projection matrix (not the full orthogonal projection matrix, but the components_.T like in PCA)
    v: vector of shape (n,)
    
    Returns:
        norm_in_projection_space
        norm_in_null_space
    """
    # Ensure vector is a numpy array and has correct shape
    v = np.asarray(v).flatten()
    
    # Step 1: Get orthonormal basis for column space of P
    Q, _ = np.linalg.qr(P)
    
    # Step 2: Get orthonormal basis for null space of P
    N = null_space(P.T)  # null space of P.T is orthogonal complement of range(P)
    
    # Step 3: Project vector onto each space
    proj_col = Q @ (Q.T @ v)
    proj_null = N @ (N.T @ v)
    
    # Step 4: Compute norms
    norm_col = np.linalg.norm(proj_col)
    norm_null = np.linalg.norm(proj_null)
    
    return norm_col, norm_null


In [78]:
f_14_normalized = output_the_mean(faults_path,14,1)
f_3_normalized = output_the_mean(faults_path,3,1)
print(f_14_normalized[:30])
print(f_3_normalized[:30])

[ 0.03304931  0.01447655 -0.00613893 -0.00968497  0.00605705  0.19944701
  0.00272531  0.42635059 -0.00940727  0.00669853 -0.00449261  0.0206242
 -0.01857882 -0.0090157   0.0257016  -0.03624111 -0.01107958  0.03310386
 -0.15795696 -0.02513252  0.01644652  0.02179586  0.02248981  0.01258481
 -0.03643698 -0.35672688 -0.03020232 -0.09794802  0.33140235 -0.70488557]
[ 2.13874762e-06  9.99983718e-01  7.67720315e-06 -3.56130738e-05
 -3.48837821e-05  2.20523162e-03 -5.81781130e-05  1.09437434e-04
 -2.74241154e-05  7.28677329e-05 -4.44704832e-05 -2.15818492e-05
  2.89089348e-05 -3.02419731e-05 -5.96763774e-05 -6.33402854e-06
  5.94598975e-05 -1.37674066e-04  6.31646484e-05  3.33011487e-05
  1.72171860e-05 -5.00327207e-05 -3.56338138e-06 -3.03358677e-06
 -6.75514750e-06 -1.69959747e-04  6.18921019e-05 -2.25951785e-05
 -5.25368910e-03  6.05818234e-05]


In [96]:
print(decompose_vector_by_projection(P_npe.T,output_the_mean(faults_path,14,1)))

(np.float64(0.23510940117249055), np.float64(0.9719689138446311))
